In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from dotenv import load_dotenv
from datetime import datetime
import os
import time

load_dotenv()

# inicializa o Sellenium
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

browser = webdriver.Chrome('/usr/bin/chromedriver', options=chrome_options)

# Get Login Page
browser.get("https://sistemaweb.hubert.com.br/")
html = browser.page_source
time.sleep(2)

lgoin = browser.find_element(by=By.NAME, value="Login")
print(os.getenv("USER"))
lgoin.send_keys(os.getenv("USER"))

password = browser.find_element(by=By.NAME, value="Senha")
password.send_keys(os.getenv("PASS"))

# Submit Login
browser.find_element(by=By.CLASS_NAME, value="btn-lg").click()


time.sleep(2)

print(f"{os.getenv('FROM')}-01")
print(datetime.now().strftime("%Y-%m-%d"))
# Get Range of dates
months = pd.date_range(start=f"{os.getenv('FROM')}-01", end=datetime.now(
).strftime("%Y-%m-%d"), freq='MS').strftime("%m-%Y").tolist()
print(months)

for month in months:

    browser.get(
        f"https://sistemaweb.hubert.com.br/Financeiro/PrestacaoContas/{month}")
    time.sleep(2)

    html = browser.page_source
    print(f"get {month}.html")

    f = open(f"contashtml/{month}.html", "w")
    f.write(html)
    f.close()

browser.close()
# Clean files without report


/tmp/ipykernel_678/1171528792.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)


felipe.lambert@outlook.com
2020-04-01
2022-03-27
['04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020', '12-2020', '01-2021', '02-2021', '03-2021', '04-2021', '05-2021', '06-2021', '07-2021', '08-2021', '09-2021', '10-2021', '11-2021', '12-2021', '01-2022', '02-2022', '03-2022']
get 04-2020.html
get 05-2020.html
get 06-2020.html
get 07-2020.html
get 08-2020.html
get 09-2020.html
get 10-2020.html
get 11-2020.html
get 12-2020.html
get 01-2021.html
get 02-2021.html
get 03-2021.html
get 04-2021.html
get 05-2021.html
get 06-2021.html
get 07-2021.html
get 08-2021.html
get 09-2021.html
get 10-2021.html
get 11-2021.html
get 12-2021.html
get 01-2022.html
get 02-2022.html
get 03-2022.html


In [15]:

import pandas as pd
from datetime import datetime
import os

from bs4 import BeautifulSoup

# Get Range of dates
months = pd.date_range(start=f"{os.getenv('FROM')}-01", end=datetime.now(
).strftime("%Y-%m-%d"), freq='MS').strftime("%m-%Y").tolist()
print(months)

# Clean files without reports
for month in list(months):
    try:
        f = open(f"contashtml/{month}.html", "r")
        bs = BeautifulSoup(f.read(), 'html.parser')
        if (not bs.find("div", "clsReportSheet")):
            months.remove(month)
            os.remove(f"contashtml/{month}.html")
    except IOError:
        print("File not accessible")
        months.remove(month)
    finally:
        f.close()
print(months)


['04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020', '12-2020', '01-2021', '02-2021', '03-2021', '04-2021', '05-2021', '06-2021', '07-2021', '08-2021', '09-2021', '10-2021', '11-2021', '12-2021', '01-2022', '02-2022', '03-2022']


Exception ignored in: <function ZipFile.__del__ at 0x7fcfa03ede10>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/zipfile.py", line 1808, in __del__
    self.close()
  File "/usr/local/lib/python3.10/zipfile.py", line 1825, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


File not accessible
File not accessible
['04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', '11-2020', '12-2020', '01-2021', '02-2021', '03-2021', '04-2021', '05-2021', '06-2021', '07-2021', '08-2021', '09-2021', '10-2021', '11-2021', '12-2021', '01-2022']


In [21]:
import re
import numpy as np


def readReceita(rec):
    table = rec.findNext("table")
    tableStr = table.prettify()
    tableStr = tableStr.replace("colspan=\"5\"", "")
    df = pd.read_html(tableStr, thousands='.', decimal=',')
    df = df[0]
    df.insert(0, "Conta", account, allow_duplicates=True)
    df.insert(0, "RecDesp", recdesp, allow_duplicates=True)
    df.insert(2, "Mes", date, allow_duplicates=True)
    df.drop(df.tail(1).index, inplace=True)
    df.rename(columns={'Unnamed: 0': 'Nome'}, inplace=True)
    return df


def readDespesa(rec):
    table = rec.findNext("table")
    tableStr = table.prettify()
    tableStr = tableStr.replace("colspan=\"5\"", "")
    df = pd.read_html(tableStr, thousands='.', decimal=',')
    df = df[0]
    df.insert(0, "Conta", account, allow_duplicates=True)
    df.insert(0, "RecDesp", recdesp, allow_duplicates=True)
    df.insert(2, "Mes", date, allow_duplicates=True)
    df.rename(columns={0: 'DespesaId', 2: 'Descricao',
              3: 'Valor', 4: 'Total', 5: 'Porcentagem'}, inplace=True)
    df.drop(columns=[1], inplace=True)
    categoria = ''
    categorias = []
    indexToRemove = []
    for index, row in df.iterrows():
        if type(row['DespesaId']) == str and not row['DespesaId'].isnumeric():
            categoria = row['DespesaId']
            indexToRemove.append(index)
        row['Categoria'] = categoria
        categorias.append(categoria)
    # Add coluna Tipo
    df.insert(3, "Tipo", categorias, allow_duplicates=True)
    # remove linha do Tipo
    df.drop(indexToRemove, axis=0, inplace=True)
    # Remove ultima linha total
    df.drop(df.tail(1).index, inplace=True)
    # df.to_csv(f'temp-{account}.csv')
    return df

def readResumo(rec):
    table = rec.findNext("table")
    tableStr = table.prettify()
    tableStr = tableStr.replace("colspan=\"5\"", "")
    df = pd.read_html(tableStr, thousands='.', decimal=',')
    df = df[0]
    df.insert(0, "Conta", account, allow_duplicates=True)
    df.insert(0, "RecDesp", recdesp, allow_duplicates=True)
    df.insert(2, "Mes", date, allow_duplicates=True)
    df.rename(columns={0: 'Nome', 1: 'Valor'}, inplace=True)
    df = df[df["Nome"].str.contains("SALDO") == False]
    # print(df)
    return df

def readFinal(rec):    
    table = rec.findNext("table")
    tableStr = table.prettify()
    tableStr = tableStr.replace("colspan=\"5\"", "")
    df = pd.read_html(tableStr, thousands='.', decimal=',')
    df = df[0]
    df.insert(0, "Mes", date, allow_duplicates=True)
    df.rename(columns={0: 'Nome', 1: 'Valor'}, inplace=True)
    
    df = df[df["Nome"].str.contains("SALDO ACUMULADO") == True]
    # print(df)
    return df

receita = []
despesa = []
resumo = []
conta = []


with pd.ExcelWriter('output.xlsx') as writer:
    
    for month in months:

        date = datetime.strptime(month,'%m-%Y').date()
        f = open(f"contashtml/{month}.html", "r")
        bs = BeautifulSoup(f.read(), 'html.parser')
        reportSheet = bs.find("div", "clsReportSheet")
        headers = reportSheet.find_all("div", "clsAccountHeader")
        accounts = []

        for header in headers:
            # print(header.find("span").text)
            headerValue = header.find("span").text
            account = re.search("^[0-9]*", headerValue).group()
            if account != '':
                recdesp = re.findall("[\w]+", headerValue)[1]

                accounts.append({'Id': account, "Nome": headerValue.replace(f'{account} - {recdesp}', '')})

                if(recdesp == "RECEITAS"):
                    receita.append(readReceita(header))
                if (recdesp == "DESPESAS"):
                    despesa.append(readDespesa(header))
                if (recdesp == "RESUMO"):
                    resumo.append(readResumo(header))
            if "RESUMO FINAL CONTA CORRENTE" == headerValue:
                conta.append(readFinal(header))
    dfReceita = pd.concat(receita)
    dfDespesa = pd.concat(despesa)
    dfResumo = pd.concat(resumo)
    dfConta = pd.concat(conta)

    dfReceita.to_excel(writer, sheet_name="Receita", index=False)
    dfDespesa.to_excel(writer, sheet_name="Despesa", index=False)
    dfResumo.to_excel(writer, sheet_name="Resumo", index=False)
    dfConta.to_excel(writer, sheet_name="Saldo", index=False)

    seen_acounts = set()
    new_list = []
    for obj in accounts:
        if obj['Id'] not in seen_acounts:
            new_list.append(obj)
            seen_acounts.add(obj['Id'])
    dfAccounts = pd.DataFrame(new_list)
    dfAccounts.to_excel(writer, sheet_name="Contas", index=False)
    # print(dfAccounts)



  RecDesp Conta         Mes                            Nome     Valor
0  RESUMO     1  2020-04-01  RECEITAS REALIZADAS CONDOMINIO      0.00
1  RESUMO     1  2020-04-01    DESPESAS CONF. DEMONSTRATIVO -16736.59
          Mes                                   Nome     Valor
1  2020-04-01  SALDO ACUMULADO DEVEDOR EM 09/05/2020 -16736.59
  RecDesp Conta         Mes                            Nome      Valor
1  RESUMO     1  2020-05-01  RECEITAS REALIZADAS CONDOMINIO  134434.99
2  RESUMO     1  2020-05-01                      REC.MULTAS      30.16
3  RESUMO     1  2020-05-01    DESPESAS CONF. DEMONSTRATIVO -112959.60
  RecDesp Conta         Mes                            Nome    Valor
0  RESUMO    12  2020-05-01                      REC.MULTAS     1.22
1  RESUMO    12  2020-05-01  RECEITAS REALIZADAS CONDOMINIO  5430.15
2  RESUMO    12  2020-05-01    DESPESAS CONF. DEMONSTRATIVO     0.00
          Mes                                  Nome     Valor
2  2020-05-01  SALDO ACUMULADO CREDOR EM 0